# ECE 176: Fine-Grained Classification Using a CB-ViT Model

The focus of our final project will be reimplmenenting the work of Shuo Zhu, Xukang Zhang, Yu Wang, Zhongyang Wang, and Jiahao Sun. The main result of this paper is the introduction of a CB-ViT model. This model combines the local feature extraction of Convolutional networks with the broad feature extraction of Vision Transformers. A version of their reseach paper can be found here: https://ietresearch.onlinelibrary.wiley.com/doi/full/10.1049/ipr2.13295. 

In [1]:
# Imports - reused from assignment 5

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data import sampler

import torchvision.datasets as dset
import torchvision.transforms as T

import numpy as np

import torch.nn.functional as F  # useful stateless functions

In [2]:
# Data setup - reused from assignment 5

NUM_TRAIN = 49000
batch_size= 64

# The torchvision.transforms package provides tools for preprocessing data
# and for performing data augmentation; here we set up a transform to
# preprocess the data by subtracting the mean RGB value and dividing by the
# standard deviation of each RGB value; we've hardcoded the mean and std.

#===========================================================================#
# You should try changing the transform for the training data to include    #
# data augmentation such as RandomCrop and HorizontalFlip                    #
# when running the final part of the notebook where you have to achieve     #
# as high accuracy as possible on CIFAR-100.                                #
# Of course you will have to re-run this block for the effect to take place #
#===========================================================================#
train_transform = transform = T.Compose([
                T.ToTensor(),
                T.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))
            ])

# We set up a Dataset object for each split (train / val / test); Datasets load
# training examples one at a time, so we wrap each Dataset in a DataLoader which
# iterates through the Dataset and forms minibatches. We divide the CIFAR-100
# training set into train and val sets by passing a Sampler object to the
# DataLoader telling how it should sample from the underlying Dataset.
cifar100_train = dset.CIFAR100('./datasets/cifar100', train=True, download=True,
                             transform=train_transform)
loader_train = DataLoader(cifar100_train, batch_size=batch_size, num_workers=2,
                          sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cifar100_val = dset.CIFAR100('./datasets/cifar100', train=True, download=True,
                           transform=transform)
loader_val = DataLoader(cifar100_val, batch_size=batch_size, num_workers=2, 
                        sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN, 50000)))

cifar100_test = dset.CIFAR100('./datasets/cifar100', train=False, download=True, 
                            transform=transform)
loader_test = DataLoader(cifar100_test, batch_size=batch_size, num_workers=2)

In [3]:
# Dtype and device selection - reused from assignment 5

USE_GPU = True
num_class = 100
dtype = torch.float32 # we will be using float throughout this tutorial

if USE_GPU and torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# Constant to control how frequently we print train loss
print_every = 100

print('using device:', device)

using device: cuda


In [4]:
# Flatten function - reused from assignment 5

def flatten(x):
    N = x.shape[0] # read in N, C, H, W
    return x.view(N, -1)  # "flatten" the C * H * W values into a single vector per image

# We need to wrap `flatten` function in a module in order to stack it
# in nn.Sequential
class Flatten(nn.Module):
    def forward(self, x):
        return flatten(x)

In [5]:
# Random weight function - reused from assignment 5

def random_weight(shape):
    """
    Create random Tensors for weights; setting requires_grad=True means that we
    want to compute gradients for these Tensors during the backward pass.
    We use Kaiming normalization: sqrt(2 / fan_in)
    """
    if len(shape) == 2:  # FC weight
        fan_in = shape[0]
    else:
        fan_in = np.prod(shape[1:]) # conv weight [out_channel, in_channel, kH, kW]
    # randn is standard normal distribution generator. 
    w = torch.randn(shape, device=device, dtype=dtype) * np.sqrt(2. / fan_in)
    w.requires_grad = True
    return w

def zero_weight(shape):
    return torch.zeros(shape, device=device, dtype=dtype, requires_grad=True)

# create a weight of shape [3 x 5]
# you should see the type `torch.cuda.FloatTensor` if you use GPU. 
# Otherwise it should be `torch.FloatTensor`
random_weight((3, 5))

tensor([[ 0.6824, -0.6195,  0.8871,  0.4586,  1.1516],
        [ 0.4421, -0.7937, -0.6834,  1.1514,  0.6247],
        [ 1.1324, -0.3626, -1.0537, -1.0352,  0.7300]], device='cuda:0',
       requires_grad=True)

In [6]:
# Check accuracy function - reused from assignment 5

def check_accuracy_part34(loader, model):
    if loader.dataset.train:
        print('Checking accuracy on validation set')
    else:
        print('Checking accuracy on test set')   
    num_correct = 0
    num_samples = 0
    model.eval()  # set model to evaluation mode
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)
            scores = model(x)
            _, preds = scores.max(1)
            num_correct += (preds == y).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))
    return acc

In [7]:
# Training function - reused from assignment 5

def train_part34(model, optimizer, epochs=1):
    """
    Train a model on CIFAR-10 using the PyTorch Module API.
    
    Inputs:
    - model: A PyTorch Module giving the model to train.
    - optimizer: An Optimizer object we will use to train the model
    - epochs: (Optional) A Python integer giving the number of epochs to train for
    
    Returns: The accuracy of the model
    """
    model = model.to(device=device)  # move the model parameters to CPU/GPU
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            model.train()  # put model to training mode
            x = x.to(device=device, dtype=dtype)  # move to device, e.g. GPU
            y = y.to(device=device, dtype=torch.long)

            scores = model(x)
            loss = F.cross_entropy(scores, y)

            # Zero out all of the gradients for the variables which the optimizer
            # will update.
            optimizer.zero_grad()

            # This is the backwards pass: compute the gradient of the loss with
            # respect to each  parameter of the model.
            loss.backward()

            # Actually update the parameters of the model using the gradients
            # computed by the backwards pass.
            optimizer.step()

            if (t + 1) % print_every == 0:
                print('Epoch %d, Iteration %d, loss = %.4f' % (e, t + 1, loss.item()))
                check_accuracy_part34(loader_val, model)
                print()
    return check_accuracy_part34(loader_val, model)

In [8]:
# Sample class - reused from assignment 5

class Network(nn.Module):
    def __init__(self, in_channel, channel_1, channel_2, num_classes):
        super().__init__()
        self.conv_1 = nn.Conv2d(in_channel, channel_1, (5,5), padding=2)
        nn.init.kaiming_normal_(self.conv_1.weight)
        self.conv_2 = nn.Conv2d(channel_1, channel_2, (3,3), padding=1)
        nn.init.kaiming_normal_(self.conv_2.weight)
        self.fc1 = nn.Linear(65536, num_classes)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):
        scores = None
        scores = self.fc1(flatten(F.relu(self.conv_2(F.relu(self.conv_1(x))))))
        return scores

In [9]:
# Training and validating of sample class - reused from assignment 5

learning_rate = 1e-3
channel_1 = 32
channel_2 = 64

model = None
optimizer = None

model = Network(3, channel_1, channel_2, 100)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

print_every = 100
train_part34(model, optimizer, epochs=1)

Epoch 0, Iteration 100, loss = 4.1276
Checking accuracy on validation set
Got 65 / 1000 correct (6.50)

Epoch 0, Iteration 200, loss = 4.0339
Checking accuracy on validation set
Got 103 / 1000 correct (10.30)

Epoch 0, Iteration 300, loss = 3.7682
Checking accuracy on validation set
Got 131 / 1000 correct (13.10)

Epoch 0, Iteration 400, loss = 3.5678
Checking accuracy on validation set
Got 134 / 1000 correct (13.40)

Epoch 0, Iteration 500, loss = 3.8066
Checking accuracy on validation set
Got 155 / 1000 correct (15.50)

Epoch 0, Iteration 600, loss = 3.4089
Checking accuracy on validation set
Got 152 / 1000 correct (15.20)

Epoch 0, Iteration 700, loss = 3.6586
Checking accuracy on validation set
Got 157 / 1000 correct (15.70)

Checking accuracy on validation set
Got 153 / 1000 correct (15.30)


0.153